In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
plt.style.use('ggplot')

import torch
from   torch import nn
from torch.utils.data import Dataset,DataLoader

# Generate Proxy Data and train dataset

In [103]:
class Dataset(Dataset):

    def __init__(self):
        
        n = 5
        m = 100

        def sigmoid(z):
            return 1/(1 + np.exp(-z))

        beta_true = np.array([1, 0.5, -0.5] + [0]*(n - 3)) # True Beta 
        X      = (np.random.random((m, n)) - 0.5)*10
        Y      = np.round(sigmoid(X @ beta_true + np.random.randn(m)*0.5))

        print(beta_true)
        # ---------- Generate Proxy Dataset , Convert ot Pytorch ---------- 
        self.data   =  torch.tensor(X)
        self.label  =  torch.tensor(Y)

    def __getitem__(self,index):
        return self.data[index].float(),self.label[index].float().reshape(-1)

    def __len__(self):
        return len(self.data)
    
    def get_input_dimmension(self):
        return self.data.size()[1]

In [104]:
batch_size      = 10
train_dataset   = Dataset()
test_dataset    = Dataset()

train_dataloader = DataLoader(dataset=train_dataset , batch_size=batch_size)
test_dataloader  = DataLoader(dataset=test_dataset  , batch_size=batch_size)

[ 1.   0.5 -0.5  0.   0. ]
[ 1.   0.5 -0.5  0.   0. ]


In [105]:
for i,(data,label) in enumerate(train_dataloader):
    print(f"iteration ({i+1})" + ", X_train Size :",data.size() , ", Y_train Size :",label.size())

iteration (1), X_train Size : torch.Size([10, 5]) , Y_train Size : torch.Size([10, 1])
iteration (2), X_train Size : torch.Size([10, 5]) , Y_train Size : torch.Size([10, 1])
iteration (3), X_train Size : torch.Size([10, 5]) , Y_train Size : torch.Size([10, 1])
iteration (4), X_train Size : torch.Size([10, 5]) , Y_train Size : torch.Size([10, 1])
iteration (5), X_train Size : torch.Size([10, 5]) , Y_train Size : torch.Size([10, 1])
iteration (6), X_train Size : torch.Size([10, 5]) , Y_train Size : torch.Size([10, 1])
iteration (7), X_train Size : torch.Size([10, 5]) , Y_train Size : torch.Size([10, 1])
iteration (8), X_train Size : torch.Size([10, 5]) , Y_train Size : torch.Size([10, 1])
iteration (9), X_train Size : torch.Size([10, 5]) , Y_train Size : torch.Size([10, 1])
iteration (10), X_train Size : torch.Size([10, 5]) , Y_train Size : torch.Size([10, 1])


# Build Model

In [106]:
class LogisticRegression(nn.Module):
    
    """
    input_dim --> regression features
    """
    
    def __init__(self,input_dim):
        super(LogisticRegression, self).__init__()

        self.linear  = nn.Linear(input_dim,1)  

    def forward(self, x):

        x_1    = self.linear(x)
        y_pred = torch.sigmoid(x_1) 

        return y_pred

# Training Hyper Parameters

In [112]:
class parameters():

    def __init__(self) :

        self.learning_rate = 0.001 
        self.epochs        = 1000
        self.lambd         = 0.1
        self.criterion     = nn.BCELoss()

parameter = parameters()

In [113]:
model     = LogisticRegression(input_dim=train_dataset.get_input_dimmension())
optimizer = torch.optim.SGD(model.parameters(),lr=parameter.learning_rate,weight_decay=parameter.lambd)

# Traing

In [114]:
for epoch in range(parameter.epochs):

    for i,(data,label) in enumerate(train_dataloader):
      
        y_pred = model(data)
        loss   = parameter.criterion(y_pred,label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 100 == 0:
        print('epoch {}, loss {}'.format(epoch+batch_size, loss.item()))

epoch 10, loss 0.5810773372650146
epoch 110, loss 0.29088208079338074
epoch 210, loss 0.22094163298606873
epoch 310, loss 0.19021907448768616
epoch 410, loss 0.17336265742778778
epoch 510, loss 0.16301816701889038
epoch 610, loss 0.15624071657657623
epoch 710, loss 0.15161103010177612
epoch 810, loss 0.14835767447948456
epoch 910, loss 0.14602549374103546


In [115]:
list(model.parameters())[0]

Parameter containing:
tensor([[ 1.0440,  0.3621, -0.3835, -0.0032,  0.0376]], requires_grad=True)

In [116]:
with torch.no_grad():

    list_accuracy = []
    print("-"*9 ,"Testing Result","-"*9)
    print("-"*35)
    for i,(data,label) in enumerate(test_dataloader):

        y_predicted = model(data)
        y_predicted_cls = y_predicted.round()
        acc = y_predicted_cls.eq(label).sum() / float(label.shape[0])
        list_accuracy.append(acc)
        print(f"Iteration ({i+1})  Accuracy = {acc: .4f}")
    
    print("-"*35)
    print("Mean Accuracy = ",np.mean(list_accuracy))

--------- Testing Result ---------
-----------------------------------
Iteration (1)  Accuracy =  1.0000
Iteration (2)  Accuracy =  1.0000
Iteration (3)  Accuracy =  1.0000
Iteration (4)  Accuracy =  0.9000
Iteration (5)  Accuracy =  1.0000
Iteration (6)  Accuracy =  1.0000
Iteration (7)  Accuracy =  0.9000
Iteration (8)  Accuracy =  1.0000
Iteration (9)  Accuracy =  1.0000
Iteration (10)  Accuracy =  1.0000
-----------------------------------
Mean Accuracy =  0.98
